In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd
import pyarrow.parquet as pq

In [2]:
type_col = {'region_name': 'category', 'city_name': 'category', 'cpe_manufacturer_name': 'category', 'cpe_model_name': 'category',
            'url_host': 'object', 'cpe_type_cd':'category', 'cpe_model_os_type': 'category', 'price': 'float64', 'date': 'object',
            'part_of_day': 'category', 'request_cnt': 'int8', 'user_id': 'int32', 'is_male':'category', 'group':'category'}

df = dd.read_parquet('data/train_data_pqt.parquet', columns=['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
                                                             'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'price', 'date',
                                                             'part_of_day', 'request_cnt', 'user_id', 'is_male', 'group'])
#df = pq.read_table('data/train_data_pqt.parquet',columns=['user_id','request_cnt','url_host'])

print(df.columns)

Index(['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
       'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'price', 'date',
       'part_of_day', 'request_cnt', 'user_id', 'is_male', 'group'],
      dtype='object')


In [3]:
df['stratify_column'] = df['region_name'].astype(str) +\
                        "_" + df['city_name'].astype(str) + \
                        "_" + df['cpe_manufacturer_name'].astype(str) + \
                        "_" + df['cpe_model_name'].astype(str) + \
                        "_" + df['cpe_type_cd'].astype(str) + \
                        "_" + df['cpe_model_os_type'].astype(str) + \
                        "_" + df['part_of_day'].astype(str) + \
                        "_" + df['is_male'].astype(str) + \
                        "_" + df['group'].astype(str)

stratified_sample = df.groupby('stratify_column',group_keys= False).apply(lambda row: row.sample(frac=0.4),
                                                                          meta = {'region_name': 'category', 'city_name':'category', 'cpe_manufacturer_name': 'category', 'cpe_model_name': 'category', 'url_host': 'object', 'cpe_type_cd':'category', 'cpe_model_os_type': 'category', 'price': 'float64', 'date': 'object', 'part_of_day': 'category', 'request_cnt': 'int8', 'user_id': 'int32', 'is_male': 'float64', 'group':'float64','stratify_column':'category'})
stratified_sample = stratified_sample.drop('stratify_column', axis = 1)


In [4]:
stratified_sample.to_parquet('stratified_sample.parquet',index=False)

MemoryError: Unable to allocate 26.7 GiB for an array with shape (210730009,) and data type <U34